In [121]:
from bs4 import BeautifulSoup
from splinter import Browser
import html5lib
import lxml

In [139]:
import regex as re
import pandas as pd

# Scrape

In [123]:
# Set up Splinter
browser = Browser('chrome')

In [124]:
# Visit the Quotes to Scrape site
url = 'https://www.lowes.com/pl/Top-load-washers-Washing-machines-Washers-dryers-Appliances/4294857976'
browser.visit(url)

# Parse

In [125]:
# Parse the HTML
html = browser.html
html_soup = BeautifulSoup(html, 'html.parser')

In [127]:
# Scrape all the washer information on a single page
washer_info = html_soup.find_all(class_ = 'sc-o9wle2-4')

In [137]:
# Initialize dictionary for washing machine information
machines = []   

# Loop through the machine listings and add their information to the dictionary
for w in washer_info:
    # Initialize dictionary of washing machine information
    washing_machine = {}

    # Find the brand(maker) of the machine
    brand_div = w.find(class_="BrandNameStyle-sc-4v6c0e-3 ifIzUE expand-product-specs").text
    washing_machine["brand"]=brand_div.replace('\xa0', '')

    # Grab the color and size of the machines
    # Size refers to the size of the drum
    description_div = w.find(class_="description-spn").text
    size_info = re.findall(r"\d+\.\d-cu ft|\d+-cu ft", description_div)
    color_info = re.findall(r"\((.*?)\)",description_div)
    washing_machine["color"]=color_info[0]
    washing_machine["size"]=size_info[0]

    # Grab the model number of the machine
    model_info = w.find(class_="tooltip-custom").text.split("#")
    washing_machine["model_#"]=model_info[1]

    # Grab the rating and the number of ratings for each machine
    rating_info = w.find(class_="RatingWrapper-sc-1djcq5w-0 cIAzCR rating size--medium read-only")
    washing_machine["rating"]=rating_info['aria-label'].split(" ")[0]
    washing_machine["number_of_ratings"]=rating_info.find(class_="rating-count").text

    # Grab whether the machine has an agitator, impeller, or both
    specs_info_2 = w.find('div', {'class': 'product-specs-styles', 'order': '2'})
    washing_machine["agitator?"] = specs_info_2.find(class_="product-specs-value").text

    # Grab whether the machine is "High Efficiency" or not
    specs_info_3 = w.find('div', {'class': 'product-specs-styles', 'order': '3'})
    washing_machine["high_efficiency"] = specs_info_3.find(class_="product-specs-value").text

    # Grab the dimensions of the machine
    specs_info_1 = w.find('div', {'class': 'product-specs-styles', 'order': '1'})
    dimen = specs_info_1.find(class_="product-specs-value").text
    d_list = re.findall(r"\d+\.\d|\d+", dimen)
    washing_machine["width"] = d_list[0]
    washing_machine["depth"] = d_list[1]
    washing_machine["height"] = d_list[2]

    # Add the washing machine dictionary to the list of machines
    machines.append(washing_machine)
    
    


In [134]:

for w in washer_info:
    specs_info_1 = w.find('div', {'class': 'product-specs-styles', 'order': '1'})
    dimen = specs_info_1.find(class_="product-specs-value").text
    d_list = re.findall(r"\d+\.\d|\d+", dimen)
    

['27.5', '27', '42']
['27', '27', '44']
['27', '28.2', '46.3']
['27.8', '27', '40.9']
['27', '27', '44']
['27', '29', '45.7']
['27.3', '27', '40.9']
['27.5', '27', '42']
['27.6', '29.4', '44.7']
['27', '28.4', '44.5']
['27', '29.3', '44']
['27.6', '29.4', '45.8']
['27.8', '27', '42.3']
['27.8', '27', '42.3']
['27.9', '28', '46']
['27', '29.3', '44']
['27.3', '27.9', '41.6']
['27', '28.4', '44.5']
['27.5', '27', '44']
['27.3', '27.9', '42.9']
['27.3', '27.9', '42.9']
['27', '28.4', '44.5']
['27.5', '27.5', '42']
['27.3', '27.9', '42.5']


# Create dataframe

In [141]:
df = pd.DataFrame(machines)
df

,brand,color,size,model_#,rating,number_of_ratings,agitator?,high_efficiency,width,depth,height
0,Whirlpool,White,3.5-cu ft,WTW4816FW,4.5,15809,Agitator,Yes,27.5,27,42
1,GE,White,4.5-cu ft,GTW465ASNWW,4.5,24640,Agitator,Yes,27,27,44
2,Midea,Magnetic Gray,4.5-cu ft,MLV45N3BMG,4,504,Impeller,Yes,27,28.2,46.3
3,Maytag,White,4.5-cu ft,MVW4505MW,4,2635,Agitator,Yes,27.8,27,40.9
4,GE,White,4.2-cu ft,GTW335ASNWW,4.5,19206,Agitator,No,27,27,44
5,Midea,White,4.5-cu ft,MLV45N3BWW,4,524,Impeller,Yes,27,29,45.7
6,Maytag,White,4.8-cu ft,MVW5430MW,4,1516,Impeller,Yes,27.3,27,40.9
7,Amana,White,3.5-cu ft,NTW4516FW,4.5,11294,Agitator,No,27.5,27,42
8,Samsung,Champagne,5.2-cu ft,WA52A5500AC/US,4.5,500,Impeller,Yes,27.6,29.4,44.7
9,LG,White,5-cu ft,WT7150CW,4.5,1703,Impeller,Yes,27,28.4,44.5


# Close the Browser

In [120]:
browser.quit()

# Export the dataset